# Testing the model data
We'll import the `model.py` file and test the different methods we have created.

In [ ]:
import model

------
<i>*Tip*: when a model is loaded in the current instance of the notebook, re-importing the same model has no effect. <br/>
As such, if some modification would happen in the module, you would need to restart the kernel.  <br/>
**The alternative** to reload the kernel is to use `importlib` module that free and reload a given module.</i>

In [ ]:
# for debugging purpose
import importlib
importlib.reload(model)

---------
In order to test a `DataModel` instance, we'll need to pass it a `query` and its associated `asset_class`.

In [ ]:
bond_issuance = '''
    let(#amt=sum(group(amt_outstanding(currency='USD'),[year(announce_date()), month(announce_date()),cntry_of_risk()]))/1000000;) 
    get(#amt) 
    for( filter(members('LEGATRUU index'), announce_date() >= '2010-01-01') ) 
'''

m = model.DataModel(query=bond_issuance, asset_class='Fixed Income')

Let's run this instance of our DataModel that we initiated for fixed income. <br/>
Running this line will run the BQL query (initialization + data download). Note that nothing will come out from this method. 

<img src="img/04_kernel_busy.png" />

In [ ]:
m.run()

Once the model has done retrieving the dataset, the kernel should be ready and be display as per below:

<img src="img/05_kernel_ready.png" />

Now, running the `build_2dim_dataset()` method should display the data.

In [ ]:
m.build_2dim_dataset(m.data, x='Month', y='Year', v='Amount Out', calc_type='sum')

---------
Let's test the equity dataset.

In [ ]:
equity_score = '''
    let(#ret1m = (product(dropna(1.0+day_to_day_total_return(start=-1m,end=0d)))-1)*100;
        #ret1m_idx = value(#ret1m,['BWORLD Index']);
        #rel_ret1m = #ret1m - #ret1m_idx;
        #avg_rel_ret = avg(group(#rel_ret1m,[country_full_name(),gics_sector_name()]));)
    get(#avg_rel_ret)
    for( members('BWORLD Index'))
'''

n = model.DataModel(query=equity_score, asset_class='Equity')
n.run()

In [ ]:
m.build_2dim_dataset(n.data, x='Sector', y='Country', v='1m return', calc_type='median')